In [103]:
import gymnasium as gym
from gymnasium import spaces
# from stable_baselines3 import PPO
# from scipy.optimize import minimize, Bounds, LinearConstraint
import plotly.graph_objs as go
import pandas as pd
import requests
import numpy as np
import yfinance as yf
import matplotlib
%matplotlib inline
import random
import plotly.io as pio
# import cvxpy as cp
# import matplotlib.pyplot as plt
import datetime as dt
# from prophet import Prophet
from sklearn.metrics import r2_score, mean_absolute_error
# from stable_baselines3.common.vec_env import DummyVecEnv
# import torch
from flipside import Flipside
from dune_client.client import DuneClient

import os
from dotenv import load_dotenv

import datetime as dt
from datetime import timedelta
import pytz  # Import pytz if using timezones

import matplotlib.pyplot as plt

from plotly.subplots import make_subplots
import streamlit as st



In [104]:
@st.cache_data(ttl=timedelta(days=7))
def fetch_and_process_tbill_data(api_url, data_key, date_column, value_column, date_format='datetime'):
    api_key = os.getenv("FRED_API_KEY")
    api_url_with_key = f"{api_url}&api_key={api_key}"

    response = requests.get(api_url_with_key)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[data_key])
        
        if date_format == 'datetime':
            df[date_column] = pd.to_datetime(df[date_column])
        
        df.set_index(date_column, inplace=True)
        df[value_column] = df[value_column].astype(float)
        return df
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame in case of failure
    
three_month_tbill_historical_api = "https://api.stlouisfed.org/fred/series/observations?series_id=TB3MS&file_type=json"

try:
    three_month_tbill = fetch_and_process_tbill_data(three_month_tbill_historical_api, "observations", "date", "value")
    three_month_tbill['decimal'] = three_month_tbill['value'] / 100
    current_risk_free = three_month_tbill['decimal'].iloc[-1]
    print(f"3-month T-bill data fetched: {three_month_tbill.tail()}")
except Exception as e:
    print(f"Error in fetching tbill data: {e}")

2024-12-23 17:38:18.135 No runtime found, using MemoryCacheStorageManager
2024-12-23 17:38:18.143 No runtime found, using MemoryCacheStorageManager


3-month T-bill data fetched:            realtime_start realtime_end  value  decimal
date                                                  
2024-07-01     2024-12-12   2024-12-12   5.20   0.0520
2024-08-01     2024-12-12   2024-12-12   5.05   0.0505
2024-09-01     2024-12-12   2024-12-12   4.72   0.0472
2024-10-01     2024-12-12   2024-12-12   4.51   0.0451
2024-11-01     2024-12-12   2024-12-12   4.42   0.0442


In [2]:
os.chdir('..')

In [3]:
from python_scripts.utils import flipside_api_results, set_random_seed, to_time, clean_prices, calculate_cumulative_return, calculate_cagr, calculate_beta,pull_data
from sql_scripts.queries import lst_portfolio_prices,eth_btc_prices,dao_advisor_portfolio, yield_portfolio_prices,token_classifier,token_prices, model_flows
from models.training import train_model

2024-12-23 15:43:57.428 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [4]:
load_dotenv()
flipside_api_key = os.getenv("FLIPSIDE_API_KEY")

In [5]:
network = 'gnosis'
days = 60

In [6]:
def token_classifier_portfolio(api_key, network='gnosis',days=60,backtest_period=4380,use_cached_data=False):
    print(f'use_cached_data: {use_cached_data}')
    classifier_path = 'data/classifier_results.csv'
    portfolio_path = 'data/classifier_prices.csv'

    if use_cached_data:
        data_struct = {'classifier':pd.read_csv(classifier_path).dropna(),
                          'portfolio':pd.read_csv(portfolio_path).dropna()}

    else:
        classifier = token_classifier(network,days)
        tokens = flipside_api_results(classifier,api_key)
        
        tokens['latest_hour'] = pd.to_datetime(tokens['latest_hour'])
        latest_hour = tokens['latest_hour'].max() 

        data_start = latest_hour - timedelta(hours=backtest_period) # 6 Months backtesting
        data_start_str = str(data_start.tz_localize(None))

        portfolio = tokens['token_address'].unique()
        token_prices_query = token_prices(portfolio,network,data_start_str)

        tokens_df = flipside_api_results(token_prices_query,api_key)

        tokens.to_csv(classifier_path,index=False)
        tokens_df.to_csv(portfolio_path,index=False)

        data_struct = {'classifier':tokens,
                            'portfolio':tokens_df}

    return data_struct



    

In [7]:
# gnosis_data = token_classifier_portfolio(
#     network='gnosis',
#     days=60,
#     api_key = flipside_api_key,
#     use_cached_data=True
# )

In [8]:
dpi_query = token_prices({'0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b'},'ethereum','2024-01-01 00:00:00')
dpi_data = flipside_api_results(dpi_query,flipside_api_key)
dpi_data_df =  clean_prices(dpi_data)
dpi_data_df.set_index('hour',inplace=True)

Beginning: '2024-01-01 00:00:00'


2024-12-23 15:43:59.088 
  command:

    streamlit run e:\Projects\steth_optimizer\steth_optimizer\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-23 15:43:59.090 No runtime found, using MemoryCacheStorageManager


Polling error. Status: 502, Response: <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>

cleaning prices
cleaned prices:                           hour   DPI_Price
0     2024-01-01T00:00:00.000Z  102.159496
1     2024-01-01T01:00:00.000Z  103.011036
2     2024-01-01T02:00:00.000Z  102.852327
3     2024-01-01T03:00:00.000Z  103.644036
4     2024-01-01T04:00:00.000Z  104.205670
...                        ...         ...
8583  2024-12-23T15:00:00.000Z  144.780000
8584  2024-12-23T16:00:00.000Z  143.196674
8585  2024-12-23T17:00:00.000Z  148.670760
8586  2024-12-23T18:00:00.000Z  146.770432
8587  2024-12-23T19:00:00.000Z  146.393381

[8588 rows x 2 columns]


In [9]:
link_query = token_prices({'0x514910771AF9Ca656af840dff83E8264EcF986CA'},'ethereum','2024-11-01 00:00:00')
link_data = flipside_api_results(link_query,flipside_api_key)
link_data_df =  clean_prices(link_data)


Beginning: '2024-11-01 00:00:00'
Query not completed. Retrying in 30 seconds...
cleaning prices
cleaned prices:                           hour  LINK_Price
0     2024-11-01T00:00:00.000Z   11.400000
1     2024-11-01T01:00:00.000Z   11.410000
2     2024-11-01T02:00:00.000Z   11.410000
3     2024-11-01T03:00:00.000Z   11.360000
4     2024-11-01T04:00:00.000Z   11.370000
...                        ...         ...
1263  2024-12-23T15:00:00.000Z   22.600000
1264  2024-12-23T16:00:00.000Z   22.162033
1265  2024-12-23T17:00:00.000Z   22.999828
1266  2024-12-23T18:00:00.000Z   22.598208
1267  2024-12-23T19:00:00.000Z   22.310127

[1268 rows x 2 columns]


In [10]:
uni_query = token_prices({'0x1f9840a85d5af5bf1d1762f925bdaddc4201f984'},'ethereum','2024-11-01 00:00:00')
uni_data = flipside_api_results(uni_query,flipside_api_key)
uni_data_df =  clean_prices(uni_data)


Beginning: '2024-11-01 00:00:00'
Query not completed. Retrying in 30 seconds...
cleaning prices
cleaned prices:                           hour  UNI_Price
0     2024-11-01T00:00:00.000Z   7.650000
1     2024-11-01T01:00:00.000Z   7.600000
2     2024-11-01T02:00:00.000Z   7.660000
3     2024-11-01T03:00:00.000Z   7.650000
4     2024-11-01T04:00:00.000Z   7.730000
...                        ...        ...
1263  2024-12-23T15:00:00.000Z  13.520000
1264  2024-12-23T16:00:00.000Z  13.508978
1265  2024-12-23T17:00:00.000Z  13.883604
1266  2024-12-23T18:00:00.000Z  13.541444
1267  2024-12-23T19:00:00.000Z  13.487293

[1268 rows x 2 columns]


In [11]:
uni_data_df.set_index('hour',inplace=True)

In [12]:
link_data_df.set_index('hour',inplace=True)

In [13]:
from python_scripts.utils import calculate_beta, calculate_cagr

In [14]:
# current_risk_free = 0.0425

In [15]:
combined_prices = pd.merge(
    link_data_df,
    dpi_data_df,
    left_index=True,
    right_index=True,
    how='inner'
)

combined_prices = combined_prices.merge(
    uni_data_df,
    left_index=True,
    right_index=True,
    how='inner'
)

In [16]:
combined_prices.index = pd.to_datetime(combined_prices.index)

In [17]:
combined_prices.to_csv('data/combined_prices.csv')

In [85]:
dpi_data_df.index = pd.to_datetime(dpi_data_df.index).tz_localize(None)

In [87]:
model_norm = pd.read_csv('data/nom_comp.csv')
model_norm.set_index('Unnamed: 0',inplace=True)
model_norm.index = pd.to_datetime(model_norm.index)
model_norm = model_norm.merge(
    dpi_data_df,
    left_index=True,
    right_index=True,
    how='inner'
)

In [88]:
model_norm 

,arbitrum_classifier Portfolio Value,WBTC_Price,UNI_Price,LINK_Price,CRV_Price,DPI_Price
2024-12-22 19:00:00,100.000000,100.000000,100.000000,100.000000,100.000000,142.840000
2024-12-22 20:00:00,100.543372,100.439533,99.220963,100.631769,98.609675,144.550000
2024-12-22 21:00:00,97.718670,99.138810,96.671388,97.788809,96.284985,141.400000
2024-12-22 22:00:00,98.915852,99.843324,98.229462,98.962094,98.427796,142.780000
2024-12-22 23:00:00,100.140296,99.727658,98.866856,100.225632,99.400099,144.760000
2024-12-23 00:00:00,99.465864,99.823346,98.229462,99.548736,97.795121,143.430000
2024-12-23 01:00:00,99.330096,99.479501,98.087819,99.413357,98.061877,144.020000
2024-12-23 02:00:00,98.413204,98.680350,96.954674,98.510830,98.333796,142.000000
2024-12-23 03:00:00,99.526408,98.909580,97.167139,99.684116,98.423835,143.740000
2024-12-23 04:00:00,99.813495,98.883292,97.025496,100.000000,98.793716,143.740000


In [94]:
def calculate_df_betas(df):
    betas = {}

    for col in df.columns:
        if col != 'DPI_Price':
            beta = calculate_beta(df, columnx='DPI_Price', columny=col)
            betas[col] = beta

    # Convert betas dictionary to a DataFrame
    betas_df = pd.DataFrame.from_dict(betas, orient='index', columns=['Beta'])
    return betas_df


In [95]:
portfolio_beta = calculate_df_betas(model_norm)
portfolio_beta

,Beta
arbitrum_classifier Portfolio Value,0.638886
WBTC_Price,0.209205
UNI_Price,0.707135
LINK_Price,0.639035
CRV_Price,0.705910


In [96]:
def calculate_cagr_for_all_columns(df):
    """
    Calculate the CAGR for each column in a DataFrame.

    Parameters:
    - df: Pandas DataFrame with datetime index and columns containing historical data.

    Returns:
    - Pandas Series with CAGR for each column.
    """
    cagr_results = {}
    for column in df.columns:
        if column != 'DPI_Price':
            cagr_results[column] = calculate_cagr(df[column].dropna())
    
    return pd.Series(cagr_results, name="CAGR")

In [ ]:
def calculate_daily_return(data):
    return data['price'].pct_change()

In [111]:
three_month_tbill[['decimal']]

,decimal
date,
1934-01-01,0.0072
1934-02-01,0.0062
1934-03-01,0.0024
1934-04-01,0.0015
1934-05-01,0.0016
...,...
2024-07-01,0.0520
2024-08-01,0.0505
2024-09-01,0.0472


In [113]:
tbill_hourly_rate = (1 + three_month_tbill[['decimal']]) ** (1 / 8760) - 1
tbill_hourly_rate

,decimal
date,
1934-01-01,8.189734e-07
1934-02-01,7.055778e-07
1934-03-01,2.736444e-07
1934-04-01,1.711046e-07
1934-05-01,1.825025e-07
...,...
2024-07-01,5.786902e-06
2024-08-01,5.624016e-06
2024-09-01,5.264847e-06


In [116]:
aligned_tbill = tbill_hourly_rate.reindex(model_norm.index, method='ffill')

hourly_excess_return = model_norm.pct_change().dropna() - aligned_tbill['decimal'].iloc[0]

# Output hourly excess return
print(hourly_excess_return.head())

                     arbitrum_classifier Portfolio Value  WBTC_Price  \
2024-12-22 20:00:00                             0.005429    0.004390   
2024-12-22 21:00:00                            -0.028099   -0.012955   
2024-12-22 22:00:00                             0.012246    0.007101   
2024-12-22 23:00:00                             0.012374   -0.001163   
2024-12-23 00:00:00                            -0.006740    0.000955   

                     UNI_Price  LINK_Price  CRV_Price  DPI_Price  
2024-12-22 20:00:00  -0.007795    0.006313  -0.013908   0.011966  
2024-12-22 21:00:00  -0.025701   -0.028256  -0.023580  -0.021797  
2024-12-22 22:00:00   0.016112    0.011993   0.022250   0.009755  
2024-12-22 23:00:00   0.006484    0.012763   0.009873   0.013863  
2024-12-23 00:00:00  -0.006452   -0.006759  -0.016152  -0.009193  


In [140]:
average_excess_returns = hourly_excess_return.mean(axis=0)

In [128]:
portfolio_beta

,Beta
arbitrum_classifier Portfolio Value,0.638886
WBTC_Price,0.209205
UNI_Price,0.707135
LINK_Price,0.639035
CRV_Price,0.705910


In [142]:
average_excess_returns_df = average_excess_returns.to_frame('Average Excess Return')
average_excess_returns_df

,Average Excess Return
arbitrum_classifier Portfolio Value,0.000854
WBTC_Price,-0.000981
UNI_Price,-0.001618
LINK_Price,0.000971
CRV_Price,0.001239
DPI_Price,0.001168


In [217]:
def annualize_cumulative_return(cumulative_returns_df, observed_days):
    """
    Annualize cumulative returns from a given DataFrame.
    
    Parameters:
    cumulative_returns_df (pd.DataFrame): DataFrame with cumulative returns
    observed_days (int): Number of observed days in the dataset
    
    Returns:
    pd.DataFrame: DataFrame with annualized returns for each column
    """
    annualized_returns = {}

    for col in cumulative_returns_df.columns:
        cumulative_return = cumulative_returns_df[col].iloc[0]
        annualized_return = (1 + cumulative_return) ** (365 / observed_days) - 1
        annualized_returns[col] = annualized_return

    # Convert the dictionary to a DataFrame
    annualized_returns_df = pd.DataFrame(annualized_returns, index=['Annualized_Return'])
    
    return annualized_returns_df


In [218]:
observed_days = (model_norm.index[-1] - model_norm.index[0]).days
observed_days

1

In [216]:
portfolio_cum_return = calculate_cumulative_return(model_norm)
portfolio_cum_return

,arbitrum_classifier Portfolio Value,WBTC_Price,UNI_Price,LINK_Price,CRV_Price,DPI_Price
Cumulative_Return,0.01676,-0.022797,-0.038952,0.019404,0.024383,0.024877


In [220]:
portfolio_annualized_return = annualize_cumulative_return(portfolio_cum_return, observed_days)
print(portfolio_annualized_return)


                   arbitrum_classifier Portfolio Value  WBTC_Price  UNI_Price  \
Annualized_Return                           430.281655   -0.999779  -0.999999   

                    LINK_Price    CRV_Price    DPI_Price  
Annualized_Return  1111.918407  6586.197524  7853.772771  


In [98]:
# portfolio_cagr = calculate_cagr_for_all_columns(model_norm)
# portfolio_cagr

cagr history: 2024-12-22 19:00:00    100.000000
2024-12-22 20:00:00    100.543372
2024-12-22 21:00:00     97.718670
2024-12-22 22:00:00     98.915852
2024-12-22 23:00:00    100.140296
2024-12-23 00:00:00     99.465864
2024-12-23 01:00:00     99.330096
2024-12-23 02:00:00     98.413204
2024-12-23 03:00:00     99.526408
2024-12-23 04:00:00     99.813495
2024-12-23 05:00:00    104.011115
2024-12-23 06:00:00    103.904420
2024-12-23 07:00:00    102.662267
2024-12-23 08:00:00    100.668075
2024-12-23 09:00:00    101.329259
2024-12-23 10:00:00    102.865003
2024-12-23 11:00:00    103.851369
2024-12-23 12:00:00    103.718104
2024-12-23 13:00:00    103.880903
2024-12-23 15:00:00    101.697674
2024-12-23 16:00:00    100.215979
2024-12-23 17:00:00     99.819788
2024-12-23 18:00:00    103.442863
2024-12-23 19:00:00    101.676017
Name: arbitrum_classifier Portfolio Value, dtype: float64
cagr history: 2024-12-22 19:00:00    100.000000
2024-12-22 20:00:00    100.439533
2024-12-22 21:00:00     99.138

arbitrum_classifier Portfolio Value     432.077495
WBTC_Price                               -0.999780
UNI_Price                                -1.000000
LINK_Price                             1117.278420
CRV_Price                              6625.988772
Name: CAGR, dtype: float64

In [18]:
link_beta = calculate_beta(combined_prices,columnx='DPI_Price',columny='LINK_Price')

In [19]:
uni_beta = calculate_beta(combined_prices,columnx='DPI_Price',columny='UNI_Price')

In [20]:
link_beta

0.21945061954709932

In [21]:
from chart_builder.scripts.visualization_pipeline import visualization_pipeline
from chart_builder.scripts.utils import main as chartBuilder

sys.path: ['C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\brandyns pc\\AppData\\Local\\Programs\\Python\\Python311', 'e:\\Projects\\steth_optimizer\\steth_optimizer', '', 'e:\\Projects\\steth_optimizer\\steth_optimizer\\Lib\\site-packages', 'e:\\projects\\ournetwork\\pipeline', 'e:\\projects\\pipeline_pkg', 'e:\\projects\\chart_builder_pkg', 'e:\\Projects\\steth_optimizer\\steth_optimizer\\Lib\\site-packages\\win32', 'e:\\Projects\\steth_optimizer\\steth_optimizer\\Lib\\site-packages\\win32\\lib', 'e:\\Projects\\steth_optimizer\\steth_optimizer\\Lib\\site-packages\\Pythonwin', 'e:\\projects\\chart_builder_pkg\\chart_builder\\scripts']
Current working directory: e:\Projects\portfolio_optimizers\classifier_optimizer
Current directory: e:\projects\chart_builder_pkg\chart_builder\scripts


Current working directory: e:\Projects\portfolio_optimizers\classifier_optimizer
Current directory: e:\projects\chart_builder_pkg\chart_builder\scripts


In [130]:
model_norm

,arbitrum_classifier Portfolio Value,WBTC_Price,UNI_Price,LINK_Price,CRV_Price,DPI_Price
2024-12-22 19:00:00,100.000000,100.000000,100.000000,100.000000,100.000000,142.840000
2024-12-22 20:00:00,100.543372,100.439533,99.220963,100.631769,98.609675,144.550000
2024-12-22 21:00:00,97.718670,99.138810,96.671388,97.788809,96.284985,141.400000
2024-12-22 22:00:00,98.915852,99.843324,98.229462,98.962094,98.427796,142.780000
2024-12-22 23:00:00,100.140296,99.727658,98.866856,100.225632,99.400099,144.760000
2024-12-23 00:00:00,99.465864,99.823346,98.229462,99.548736,97.795121,143.430000
2024-12-23 01:00:00,99.330096,99.479501,98.087819,99.413357,98.061877,144.020000
2024-12-23 02:00:00,98.413204,98.680350,96.954674,98.510830,98.333796,142.000000
2024-12-23 03:00:00,99.526408,98.909580,97.167139,99.684116,98.423835,143.740000
2024-12-23 04:00:00,99.813495,98.883292,97.025496,100.000000,98.793716,143.740000


In [135]:
prices_fig = visualization_pipeline(
    df=model_norm.drop(columns=['DPI_Price']),
    title='combined_prices',
    chart_type = 'line',
    ticksuffix=dict(y1=None,y2=None),
    sort_list=True,
    show_legend=True
)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: 2024-12-22 19:00:00
turn_to_time: True


In [136]:
chartBuilder(
    fig=prices_fig,
    add_the_date=True,
    save=False
)

save:False
logo path: ../img/Logos/None
Generating line plot...
axes titles at viz pipeline: {'y1': None, 'y2': None}
No GroupBy Col
cols to plot: ['WBTC_Price', 'arbitrum_classifier Portfolio Value', 'UNI_Price', 'LINK_Price', 'CRV_Price']
axes data to plot: {'y1': Index(['arbitrum_classifier Portfolio Value', 'WBTC_Price', 'UNI_Price',
       'LINK_Price', 'CRV_Price'],
      dtype='object'), 'y2': []}
tick0 in func: 2024-12-22 19:00:00
sort_list: True
Ranked columns: LINK_Price                             2432.265343
arbitrum_classifier Portfolio Value    2427.610091
CRV_Price                              2412.260091
WBTC_Price                             2385.485957
UNI_Price                              2349.929178
dtype: float64
Ranked columns: CRV_Price                              102.438260
LINK_Price                             101.940433
arbitrum_classifier Portfolio Value    101.676017
WBTC_Price                              97.720318
UNI_Price                              

In [24]:
dpi_cagr = calculate_cagr(combined_prices['DPI_Price'])

cagr history: hour
2024-11-01 00:00:00+00:00     78.080000
2024-11-01 01:00:00+00:00     78.030000
2024-11-01 02:00:00+00:00     77.490000
2024-11-01 03:00:00+00:00     77.540000
2024-11-01 04:00:00+00:00     77.980000
                                ...    
2024-12-23 15:00:00+00:00    144.780000
2024-12-23 16:00:00+00:00    143.196674
2024-12-23 17:00:00+00:00    148.670760
2024-12-23 18:00:00+00:00    146.770432
2024-12-23 19:00:00+00:00    146.393381
Name: DPI_Price, Length: 1268, dtype: float64


In [25]:
link_cagr = calculate_cagr(combined_prices['LINK_Price'])

cagr history: hour
2024-11-01 00:00:00+00:00    11.400000
2024-11-01 01:00:00+00:00    11.410000
2024-11-01 02:00:00+00:00    11.410000
2024-11-01 03:00:00+00:00    11.360000
2024-11-01 04:00:00+00:00    11.370000
                               ...    
2024-12-23 15:00:00+00:00    22.600000
2024-12-23 16:00:00+00:00    22.162033
2024-12-23 17:00:00+00:00    22.999828
2024-12-23 18:00:00+00:00    22.598208
2024-12-23 19:00:00+00:00    22.310127
Name: LINK_Price, Length: 1268, dtype: float64


In [26]:
uni_cagr = calculate_cagr(combined_prices['UNI_Price'])

cagr history: hour
2024-11-01 00:00:00+00:00     7.650000
2024-11-01 01:00:00+00:00     7.600000
2024-11-01 02:00:00+00:00     7.660000
2024-11-01 03:00:00+00:00     7.650000
2024-11-01 04:00:00+00:00     7.730000
                               ...    
2024-12-23 15:00:00+00:00    13.520000
2024-12-23 16:00:00+00:00    13.508978
2024-12-23 17:00:00+00:00    13.883604
2024-12-23 18:00:00+00:00    13.541444
2024-12-23 19:00:00+00:00    13.487293
Name: UNI_Price, Length: 1268, dtype: float64


In [159]:
average_excess_returns_df[average_excess_returns_df.index=='DPI_Price']['Average Excess Return'].values[0]

0.0011680622662884393

In [163]:
current_risk_free

0.044199999999999996

In [169]:
tbill_hourly_rate.iloc[-1].values[0]

4.93734574957827e-06

In [222]:
portfolio_cum_return

,arbitrum_classifier Portfolio Value,WBTC_Price,UNI_Price,LINK_Price,CRV_Price,DPI_Price
Cumulative_Return,0.01676,-0.022797,-0.038952,0.019404,0.024383,0.024877


In [229]:
observed_hours = observed_days * 24
observed_hours

24

In [236]:
aligned_risk_free = tbill_hourly_rate.iloc[-1] * observed_hours
aligned_risk_free = aligned_risk_free.values[0]

In [237]:
dpi_cumulative_return = portfolio_cum_return['DPI_Price'].values[0]

In [239]:
cumulative_risk_premium = dpi_cumulative_return - aligned_risk_free
cumulative_risk_premium

0.02475815455611241

In [171]:
dpi_cagr

76.38906036178085

In [172]:
cumulative_risk_premium

0.001163124920538861

In [173]:
link_beta

0.21945061954709932

In [221]:
portfolio_beta

,Beta
arbitrum_classifier Portfolio Value,0.638886
WBTC_Price,0.209205
UNI_Price,0.707135
LINK_Price,0.639035
CRV_Price,0.705910


In [245]:
portfolio_expected_return = aligned_risk_free + (portfolio_beta[portfolio_beta.index=='arbitrum_classifier Portfolio Value'].values[0]*cumulative_risk_premium)
portfolio_expected_return[0]*100

1.5936127062442247

In [174]:
expected_return = current_risk_free + (link_beta * cumulative_risk_premium)
expected_return * 100

4.445524848442292

In [175]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.colors as mcolors


def create_interactive_sml(risk_free_rate, market_risk_premium, betas, returns):
    """
    Creates an interactive Security Market Line (SML) plot with dynamic inputs.
    
    Parameters:
    - risk_free_rate (float): The risk-free rate.
    - market_risk_premium (float): The market risk premium.
    - betas (dict): Dictionary of asset betas with names as keys and beta values as values.
    - returns (dict): Dictionary of actual returns with names as keys and return values as values.
    
    Example:
    betas = {
        'RL': 0.5,
        'MVO': 0.7,
        'Historical': 0.6,
        'Defi': 1.0,
        'Non-Defi': 0.8
    }
    returns = {
        'RL': 0.08,
        'MVO': 0.10,
        'Historical': 0.09,
        'Defi': 0.12,
        'Non-Defi': 0.07
    }
    """
    def generate_shades(base_color, light_factor=1.3, dark_factor=0.7):
        rgb = mcolors.to_rgb(base_color)
        lighter_shade = mcolors.to_hex(tuple(min(1, c * light_factor) for c in rgb))
        darker_shade = mcolors.to_hex(tuple(max(0, c * dark_factor) for c in rgb))
        return lighter_shade, darker_shade

    # List of base colors to assign dynamically
    base_colors = [
        '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
        '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
    ]
    
    # Collect all beta values and filter out None
    beta_values = [beta for beta in betas.values() if beta is not None]
    
    # Determine the beta range
    max_beta = max(beta_values) if beta_values else 6
    min_beta = min(beta_values) if min(beta_values) < 0 else 0
    beta_range = np.linspace(min_beta, np.absolute(max_beta) * 1.1, 100)  # Slightly extend the range

    # Calculate expected returns for the SML line
    expected_returns = risk_free_rate + beta_range * market_risk_premium

    # Create the SML line
    sml_line = go.Scatter(
        x=beta_range,
        y=expected_returns*100,
        mode='lines',
        name='SML',
        line=dict(color='black')
    )

    # Plot points for expected and actual returns dynamically
    data = [sml_line]

    for i, (name, beta) in enumerate(betas.items()):
        if beta is not None:
            # Get base color and generate shades
            base_color = base_colors[i % len(base_colors)]
            lighter_shade, darker_shade = generate_shades(base_color)

            # Expected return based on SML (darker shade)
            expected_return = risk_free_rate + beta * market_risk_premium
            data.append(go.Scatter(
                x=[beta],
                y=[expected_return * 100],  # Convert to percentage
                mode='markers',
                marker=dict(size=10, color=darker_shade),
                name=f'{name} Expected ({expected_return:.2%})'
            ))

            # Actual return (lighter shade)
            actual_return = returns.get(name)
            if actual_return is not None:
                data.append(go.Scatter(
                    x=[beta],
                    y=[actual_return * 100],  # Convert to percentage
                    mode='markers',
                    marker=dict(size=10, color=lighter_shade),
                    name=f'{name} Actual ({actual_return:.2%})'
                ))

    # Risk-Free Rate line
    risk_free_line = go.Scatter(
        x=[min(beta_range), max(beta_range)],
        y=[risk_free_rate*100, risk_free_rate*100],
        mode='lines',
        line=dict(dash='dash', color='green'),
        name='Risk-Free Rate'
    )
    
    data.append(risk_free_line)

    # Layout settings
    layout = go.Layout(
        title='Security Market Line',
        xaxis=dict(title='Beta (Systematic Risk)'),
        yaxis=dict(title='Return',ticksuffix='%'),
        showlegend=True,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font = dict(size=18,color='black')
    )

    # Combine all the plots
    fig = go.Figure(data=data, layout=layout)
    return fig


In [176]:
portfolio_beta.to_dict()['Beta']

{'arbitrum_classifier Portfolio Value': 0.6388856943518529,
 'WBTC_Price': 0.20920522346125556,
 'UNI_Price': 0.7071346360423548,
 'LINK_Price': 0.6390345339415766,
 'CRV_Price': 0.7059097159277392}

In [177]:
betas = portfolio_beta.to_dict()['Beta']

In [178]:
average_excess_returns_df.to_dict()['Average Excess Return']

{'arbitrum_classifier Portfolio Value': 0.0008541694163857725,
 'WBTC_Price': -0.0009814958048609062,
 'UNI_Price': -0.0016181889271485715,
 'LINK_Price': 0.00097095388436069,
 'CRV_Price': 0.0012391260721093848,
 'DPI_Price': 0.0011680622662884393}

In [179]:
link_return = calculate_cumulative_return(link_data_df)

In [180]:
link_cagr

103.10585252603855

In [181]:
link_return

,LINK_Price
Cumulative_Return,0.957029


In [201]:
average_excess_returns_df

,Average Excess Return
arbitrum_classifier Portfolio Value,0.000854
WBTC_Price,-0.000981
UNI_Price,-0.001618
LINK_Price,0.000971
CRV_Price,0.001239
DPI_Price,0.001168


In [202]:
returns = average_excess_returns_df.to_dict()['Average Excess Return']


In [203]:
def calculate_sml_data(prices):
    betas = pd.DataFrame()
    cagrs = pd.DataFrame()
    for col in prices.columns:
        cagr = calculate_cagr(prices[col])
        cagrs.loc[col, 'cagr'] = cagr


    

In [205]:
cumulative_risk_premium

0.001163124920538861

In [208]:
tbill_hourly_rate.iloc[-1].values[0]

4.93734574957827e-06

In [215]:
returns

{'arbitrum_classifier Portfolio Value': 0.0008541694163857725,
 'WBTC_Price': -0.0009814958048609062,
 'UNI_Price': -0.0016181889271485715,
 'LINK_Price': 0.00097095388436069,
 'CRV_Price': 0.0012391260721093848,
 'DPI_Price': 0.0011680622662884393}

In [213]:
betas

{'arbitrum_classifier Portfolio Value': 0.6388856943518529,
 'WBTC_Price': 0.20920522346125556,
 'UNI_Price': 0.7071346360423548,
 'LINK_Price': 0.6390345339415766,
 'CRV_Price': 0.7059097159277392}

In [209]:
fig = create_interactive_sml(
                tbill_hourly_rate.iloc[-1].values[0],
                cumulative_risk_premium,
                betas,
                returns
    )

In [210]:
fig.show()

In [43]:
target = pd.read_csv('data/new_compositions.csv')

In [44]:
target.drop(columns='Unnamed: 0',inplace=True)

In [45]:
reshaped_df = target.melt(
    var_name="token",        # Name of the new column for tokens
    value_name="composition" # Name of the new column for compositions
)

reshaped_df

,token,composition
0,CRV,0.000000
1,LINK,0.940936
2,UNI,0.059064
3,WBTC,0.000000


In [46]:
model_fig6 = visualization_pipeline(
            df=reshaped_df,
            groupby='token',
            num_col='composition',
            title='viz_port_values',
            chart_type='pie',
            show_legend=False,
            sort_list = False,
            line_width=0,
            legend_placement=dict(x=0.1,y=1.3),
            margin=dict(t=150,b=0,l=0,r=0),
            annotation_prefix='$',
            annotations=False,
        )



Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: min
turn_to_time: False
df sort order at init: ['CRV' 'LINK' 'UNI' 'WBTC']


In [47]:
chartBuilder(
    fig=model_fig6,
    title='Target Composition',
    dt_index=False,
    add_the_date=False,
    show=False,
    save=False
)

save:False
logo path: ../img/Logos/None
Generating pie chart...
self.textinfo: percent+label
original_labels: ['CRV' 'LINK' 'UNI' 'WBTC']
df_copy:        composition
token             
CRV       0.000000
LINK      0.940936
UNI       0.059064
WBTC      0.000000
percent+label, ['LINK    ', 'UNI    ', 'CRV    ']
textinfo: percent+label
Columns: Index(['composition', 'token'], dtype='object') 
Index: RangeIndex(start=0, stop=4, step=1)


In [48]:
from diskcache import Cache


In [49]:
model_name = 'arbitrum_classifier'

In [50]:
model_cache = Cache('test_model_cache')

classifier_data = model_cache.get(f'{model_name} Classifier')

In [51]:
portfolio = classifier_data['token_address'].unique()
portfolio

array(['0x11cdb42b0eb46d95f990bedd4695a6e3fa034978',
       '0xf97f4df75117a78c1a5a0dbb814af92458539fb4',
       '0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0',
       '0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f'], dtype=object)

In [52]:
start_date = '2024-12-20 00:00:00'

print(f'prices_start: {start_date}')



prices_start: 2024-12-20 00:00:00


In [77]:
ACCOUNT_ADDRESS = os.getenv('MODEL_ADDRESS')
ACCOUNT_ADDRESS

'0x75baD5ae9f46e8AEBf61e4A7179cEf3A0CeD6766'

In [78]:
flows_data = model_flows(portfolio,ACCOUNT_ADDRESS,'arbitrum')

In [54]:
flows_data_df = flipside_api_results(flows_data,flipside_api_key)
flows_data_df.set_index('dt',inplace=True)
flows_data_df.index = pd.to_datetime(flows_data_df.index).strftime('%Y-%m-%d')
daily_flows = flows_data_df.groupby([flows_data_df.index,'symbol','transaction_type'])[['amount_usd']].sum().reset_index().set_index('dt')
daily_flows.index = pd.to_datetime(daily_flows.index)

Query not completed. Retrying in 30 seconds...


In [74]:
flows_fig_1 = visualization_pipeline(
    df=daily_flows,
    title='flows_data_df_1',
    chart_type='bar',
    groupby='symbol',
    num_col='amount_usd',
    barmode='relative',
    show_legend=True,
    tickprefix=dict(y1='$',y2=None),
    buffer=1,
    legend_placement=dict(x=0.1,y=0.8)
)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: 2024-12-17 00:00:00
turn_to_time: True
df sort order at init: ['LINK' 'UNI']


In [75]:
chartBuilder(
    fig=flows_fig_1,
    title='Flows by Token',
    dt_index=False,
    add_the_date=False,
    show=True,
    save=False
)

save:False
logo path: ../img/Logos/None
Generating bar plot...
GroupBy Col: symbol
legend_orientation: v
cumulative_sort: False
sorted_bar_legend_orientation: v
x_range_start:2024-12-16 00:00:00
df @ rank_by_col:             amount_usd symbol
dt                           
2024-12-17  198.732670   LINK
2024-12-19  871.820572   LINK
2024-12-19   99.896439    UNI
2024-12-21  100.114082   LINK
2024-12-21   27.456997    UNI
df columns: Index(['amount_usd', 'symbol'], dtype='object')
sort_col: symbol
sort_list: ['LINK', 'UNI']
df[sort_col]:
['LINK' 'UNI']
Missing values in sort_list: set()
i_df:            amount_usd symbol
dt                           
2024-12-17  198.732670   LINK
2024-12-19  871.820572   LINK
2024-12-21  100.114082   LINK
col:amount_usd
text font size: 14
x:100.114081866
i_df:            amount_usd symbol
dt                           
2024-12-19   99.896439    UNI
2024-12-21   27.456997    UNI
col:amount_usd
text font size: 14
x:27.456996962
x_ticks:['2024-12-17', '2024-1

e:\projects\chart_builder_pkg\chart_builder\scripts\utils.py:554: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.



In [70]:
flows_fig_2 = visualization_pipeline(
    df=daily_flows,
    title='flows_data_df_1',
    chart_type='bar',
    groupby='transaction_type',
    num_col='amount_usd',
    barmode='relative',
    tickprefix=dict(y1='$',y2=None),
    buffer=1,
    text=True,
    textposition='auto'

)

Initialized axes_data: {'y1': 'All', 'y2': []}
tick0: 2024-12-17 00:00:00
turn_to_time: True
df sort order at init: ['inflow']


In [71]:
chartBuilder(
    fig=flows_fig_2,
    title='Flows by Type',
    dt_index=False,
    groupby='True',
    add_the_date=False,
    show=True,
    save=False
)

save:False
grouping by transaction_type... w/ sum
None
['date', 'dt', 'hour', 'time', 'day', 'month', 'year', 'week', 'timestamp', 'date(utc)', 'block_timestamp', 'ds', 'period', 'date_time', 'trunc_date', 'quarter', 'block_time', 'block_date', None]
col: dt
convert col to dt: dt
col: transaction_type
col: amount_usd
No specific time column found. Defaulting to daily frequency.
DatetimeIndex(['2024-12-21', '2024-12-17', '2024-12-19'], dtype='datetime64[ns]', name='dt', freq=None)
time_freq: d
self.df @ groupby:            transaction_type  amount_usd
dt                                     
2024-12-21           inflow  127.571079
2024-12-17           inflow  198.732670
2024-12-19           inflow  971.717010
logo path: ../img/Logos/None
Generating bar plot...
GroupBy Col: transaction_type
legend_orientation: v
cumulative_sort: False
sorted_bar_legend_orientation: v
x_range_start:2024-12-16 00:00:00
df @ rank_by_col:            transaction_type  amount_usd
dt                             

e:\projects\chart_builder_pkg\chart_builder\scripts\utils.py:554: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.



In [63]:
import time
import requests
from requests.exceptions import RequestException
from datetime import timedelta
import streamlit as st

In [64]:
def fetch_defillama_yield_pools():
    base_url = 'https://yields.llama.fi/pools'

    response = requests.get(base_url)

    data = response.json()

    return data

In [65]:
data = fetch_defillama_yield_pools()

In [ ]:
pool_data = pd.DataFrame(data['data'])
pool_data[pool_data['symbol'].isin(['SDAI','SUSDE'])]

,chain,project,symbol,tvlUsd,apyBase,apyReward,apy,rewardTokens,pool,apyPct1D,...,sigma,count,outlier,underlyingTokens,il7d,apyBase7d,apyMean30d,volumeUsd1d,volumeUsd7d,apyBaseInception
6,Ethereum,ethena-usde,SUSDE,3918238295,14.27796,NaN,14.27796,None,66985a81-9c51-46ca-9977-42b4fe7bc6df,0.02584,...,0.45152,310,False,None,NaN,NaN,23.94668,NaN,NaN,NaN
20,Ethereum,aave-v3,SUSDE,1140150273,0.00000,NaN,0.00000,None,29932dea-cd71-44c3-95bd-3e1525f4e3dd,0.00000,...,0.00000,179,False,[0x9D39A5DE30e57443BfF2A8307A4256c8797A3497],NaN,NaN,0.00000,NaN,NaN,NaN
115,Ethereum,pendle,SUSDE,134184494,23.69912,1.42544,25.12456,[0x808507121b80c02388fad14726482e061b8da827],bed5f04d-d921-4b88-87b6-8a9972201d29,-0.83272,...,0.20443,92,False,"[0xe00bd3df25fb187d6abbb620b3dfd19839947b81, 0...",NaN,NaN,27.01413,NaN,NaN,NaN
116,Ethereum,pendle,SUSDE,134184494,23.33397,NaN,23.33397,None,6b28892f-0909-418d-b4bb-3106fff72449,-0.78333,...,0.20421,92,False,[0x9d39a5de30e57443bff2a8307a4256c8797a3497],NaN,NaN,23.65585,NaN,NaN,NaN
171,Ethereum,morpho-blue,SUSDE,87012504,NaN,NaN,0.00000,[0x58D97B57BB95320F9a05dC918Aef65434969c2B2],c0921c0f-51aa-45b8-87e3-5eec59a6972b,NaN,...,0.41164,268,False,[0x9D39A5DE30e57443BfF2A8307A4256c8797A3497],NaN,NaN,0.00000,NaN,NaN,NaN
174,Ethereum,pendle,SUSDE,85045429,21.06018,1.46838,22.52856,[0x808507121b80c02388fad14726482e061b8da827],992d00f3-d43f-44fe-8b62-987e8610c9a8,-0.95449,...,0.35415,157,False,"[0xee9085fc268f6727d5d4293dbabccf901ffdcc29, 0...",NaN,NaN,28.04543,NaN,NaN,NaN
175,Ethereum,pendle,SUSDE,85045429,23.51730,NaN,23.51730,None,104f1152-d692-4e9f-9e79-db1dddd70891,-0.26587,...,0.28299,157,False,[0x9d39a5de30e57443bff2a8307a4256c8797a3497],NaN,NaN,28.15818,NaN,NaN,NaN
182,Ethereum,pendle,SUSDE,80783164,20.65749,4.03754,24.69503,[0x808507121b80c02388fad14726482e061b8da827],b3e2002b-c857-451c-ad63-a7ff1c0634be,-0.70210,...,0.28470,41,False,"[0xb7de5dfcb74d25c2f21841fbd6230355c50d9308, 0...",NaN,NaN,30.48992,NaN,NaN,NaN
183,Ethereum,pendle,SUSDE,80783164,19.30039,NaN,19.30039,None,606e53bf-ea88-4d4e-92b8-fcd6e97e4106,0.23052,...,0.08309,41,False,[0x9d39a5de30e57443bff2a8307a4256c8797a3497],NaN,NaN,20.60877,NaN,NaN,NaN
209,Gnosis,sdai,SDAI,69069398,NaN,NaN,14.13902,None,13392973-be6e-4b2f-bce9-4f7dd53d1c3a,-0.05608,...,0.17076,442,False,None,NaN,NaN,11.62683,NaN,NaN,NaN


In [67]:
@st.cache_data(ttl=timedelta(days=1))
def defillama_yield(pools):
    base_url = 'https://yields.llama.fi/chart/'

    # Placeholder to store data for each pool
    all_data = []

    # Loop through each pool and retrieve data
    for pool in pools:
        url = f'{base_url}{pool}'
        retries = 0
        while retries < 5:
            try:
                response = requests.get(url)
                response.raise_for_status()
                data = response.json()
                
                # Extract individual records and add pool ID to each entry
                for entry in data.get('data', []):
                    entry['pool'] = pool
                    all_data.append(entry)
                
                print(f"Data retrieved successfully for pool: {pool}")
                time.sleep(2)
                break
            except RequestException as e:
                retries += 1
                wait_time = 2 ** retries  # Exponential backoff (2, 4, 8, 16, 32 seconds)
                print(f"Error: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)

    # Convert to a DataFrame
    df = pd.DataFrame(all_data)
    
    # Convert 'timestamp' to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    return df

2024-12-23 00:52:56.785 No runtime found, using MemoryCacheStorageManager


In [68]:
# data = defillama_yield(pools)

NameError: name 'pools' is not defined